In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Chargement d'un dataset pour test rapide
from huggingface_hub import login
login("THIS_IS_MY_KEY")  # ta clé API Hugging Face ici

dataset = load_dataset("ag_news")
train_data = dataset["train"].shuffle(seed=42).select(range(2000))
eval_data = dataset["test"].select(range(500))

# Tokenisation
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_data = train_data.map(tokenize, batched=True)
eval_data = eval_data.map(tokenize, batched=True)

# Déclaration du modèle
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=4)

# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"

)

# Entraîneur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1-3898989071.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.426300,0.478391


TrainOutput(global_step=125, training_loss=0.7037408714294433, metrics={'train_runtime': 31.9786, 'train_samples_per_second': 62.542, 'train_steps_per_second': 3.909, 'total_flos': 66236061696000.0, 'train_loss': 0.7037408714294433, 'epoch': 1.0})

In [ ]:
trainer.save_model("finetuned_distilbert_agnews")


In [ ]:
import torch
text = "Tesla's CEO make much more money than ever"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

with torch.no_grad():
    output = model(**inputs)
    pred_id = torch.argmax(output.logits, dim=1).item()

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

print("🧠 Prediction ID:", pred_id)
print("🏷️ Predicted label:", label_map[pred_id])


🧠 Prediction ID: 2
🏷️ Predicted label: Business


In [ ]:
trainer.create_optimizer()
print(trainer.optimizer)


AcceleratedOptimizer (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 2e-05
    lr: 0.0
    maximize: False
    weight_decay: 0.01

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 2e-05
    lr: 0.0
    maximize: False
    weight_decay: 0.0
)
